In [1]:
import os
import re
import olefile

In [7]:
#@title Folder Structure
filename = "/home/bruno/PreguicaGigante/data/mineral/aragonita_rugosa.bsp"
path_to_save = '/home/bruno/PreguicaGigante/data/mineral/'
os.makedirs(path_to_save+'/Images',exist_ok=True)
os.makedirs(path_to_save+'/TempBin',exist_ok=True)

In [8]:
#@title Processing data streams
if olefile.isOleFile(filename):
    with olefile.OleFileIO(filename) as ole:
        streams = ole.listdir(storages=False, streams=True)

        streams_list = ole.openstream(['Spectra','IndexTable']).read()

        streams_list = re.findall(b'(\\w{8}-\\w{8})',streams_list)

#  with olefile.OleFileIO(filename) as ole:
        stream_data = []
        for stream_ in streams_list:
            stream_data.append(ole.openstream(['Spectra',stream_.decode()]).read())

for name,data in zip(streams_list,stream_data):
    with open(path_to_save+'/TempBin/'+name.decode()+'.bin','wb+') as f:
        f.write(data)

In [9]:
stream_name = streams_list[0]

jpg_byte_start = b'\xff\xd8\xff\xe0'
jpg_byte_end = b'\xff\xd9'
jpg_image = bytearray()

with open(path_to_save+'/TempBin/'+stream_name.decode()+'.bin', 'rb') as f:
    full_extracted_data = f.read()

    startimg = full_extracted_data.find(jpg_byte_start)

    pattern = r'(?:\x00SpectName.{11}\x00PropType\x01.{19}\x31\x2e\x30\x30.{20})([A-Za-z0-9_() ]+)(?:.{4}[A-Z])'
    match = re.findall(pattern.encode(), full_extracted_data)
   
    if match:
        sample_name = match[0].decode()
    else:
        print('No sample name found')
        sample_name = 'noname'
    
    #sample_name = re.findall(b'(?:\x00SpectName)(?:.{11})(?:\x00PropType\x01)(?:.{19})(?:\x31\x2e\x30\x30)(?:\W{20})(.+)(?:.{4})(?:\x53\x70)',full_extracted_data)[0].decode()

    if startimg == -1:
        print('!Warning!Could not find a valid JPEG image in the data stream:'+stream_name.decode())
        print('sample name:' + sample_name)
        print('It is ok if you did not take a visible image for this experiment.')

    else:
        print('Image found:'+stream_name.decode()+' Saving...\n')
        endimg = full_extracted_data.find(jpg_byte_end, startimg) + len(jpg_byte_end)
        jpg_image += full_extracted_data[startimg:endimg]

        with open(path_to_save+'/Images/'+stream_name.decode()+'_'+sample_name+'.jpg', 'wb+') as f:
            f.write(jpg_image)

Image found:0003A276-67506ABC Saving...



In [10]:
for filename in os.listdir(path_to_save+'TempBin'):
    file_path = os.path.join(path_to_save+'TempBin', filename)
    if os.path.isfile(file_path):
        os.remove(file_path)
    elif os.path.isdir(file_path):
        for sub_filename in os.listdir(file_path):
            sub_file_path = os.path.join(file_path, sub_filename)
            os.remove(sub_file_path)
        os.rmdir(file_path)
os.rmdir(path_to_save+'TempBin')